# **Methylation Biomarkers for Predicting Cancer**

## **Deep Learning Approaches for Cancer Classification**

**Author:** Meg Hutch

**Date:** February 14, 2020

**Objective:** Use neural networks to classify cancer type. 

**Note:** In this version, I will only test the ability of methylation levels to classify cancer types. I will not include phenotypic data for now. Additionally, this version has our data split 70% for training and 30% for testing. The 70% training data will undergo leave-one-out-cross-fold validation to tune hyperparameters prior to testing final performance on the 30% test set. 

Note: This is the new version of the script where we normalize gene counts using DEseq2 in the initial pre-processing script in R. This provided more than double the number of Principal Components that make up 90% of the variance (157). Regardless, we will begin running the deep learning classifier on the revised data. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

In [2]:
# set working directory for git hub
import os
os.chdir('/projects/p31049/Multi_Cancer_DL/')
#os.chdir('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/')
os. getcwd()

'/projects/p31049/Multi_Cancer_DL'

**Import Training, Testing, and Principal component data**

**Full Dataset**

In [3]:
# Training set
#mcTrain_x = pd.read_csv('02_Processed_Data/Final_Datasets/mcTrain_x_Full_70_30.csv')
#mcTrain_y = pd.read_csv('02_Processed_Data/Final_Datasets/mcTrain_y_Full_70_30.csv')
# Testing set
#mcTest_x = pd.read_csv('02_Processed_Data/Final_Datasets/mcTest_x_Full_70_30.csv')
#mcTest_y = pd.read_csv('02_Processed_Data/Final_Datasets/mcTest_y_Full_70_30.csv')
# Principal Components that make up 90% of the variance of the training set
#pca_Train = pd.read_csv('02_Processed_Data/Final_Datasets/pca_train_Full_70_30.csv')
# Principal Components projected onto the test set
#pca_Test = pd.read_csv('02_Processed_Data/Final_Datasets/pca_test_Full_70_30.csv')




In [4]:
#mcTrain_y.head()
#pca_Train.head()

# **Pre-Process Data**

In [5]:
# rename the first column name of the PC dataframes
pca_Train.rename(columns={'Unnamed: 0':'id'}, inplace=True)
pca_Test.rename(columns={'Unnamed: 0':'id'}, inplace=True)

**Remove genetic data from the feature sets. This is because we are replacing them with the PCs**

In [6]:
mcTrain_x = mcTrain_x[['id', 'dilute_library_concentration', 'age', 'gender', 'frag_mean']]
mcTest_x = mcTest_x[['id', 'dilute_library_concentration', 'age', 'gender', 'frag_mean']]

In [7]:
# merge PCs with clinical/phenotypic data
mcTrain_x = pd.merge(mcTrain_x, pca_Train, how="left", on="id") 
mcTest_x = pd.merge(mcTest_x, pca_Test, how="left", on="id") 

**Drop demographic data for these experiments**

In [8]:
mcTrain_x = mcTrain_x.drop(columns=["dilute_library_concentration", "age", "gender", "frag_mean"])
mcTest_x = mcTest_x.drop(columns=["dilute_library_concentration", "age", "gender", "frag_mean"])

**Convert id to index**

In [9]:
mcTrain_x = mcTrain_x.set_index('id')
mcTrain_y = mcTrain_y.set_index('id')

mcTest_x = mcTest_x.set_index('id')
mcTest_y = mcTest_y.set_index('id')

**Normalize Data**

From my reading, it seems that normalization, as opposed to standardization, is the more optimal approach when data is not normally distributed. 

Normalization will rescale our values into range of [0,1]. We need to normalize both the training and test sets

In [10]:
from sklearn.preprocessing import MinMaxScaler

# The normalization function to be performed will convert dataframe into array, for this reason we'll have to convert it back
# Thus, need to store columns and index
# select all columns
cols = list(mcTrain_x.columns.values)
index_train = list(mcTrain_x.index)
index_test = list(mcTest_x.index)

# Normalize data
scaler = MinMaxScaler()
mcTrain_x = scaler.fit_transform(mcTrain_x.astype(np.float))
mcTest_x = scaler.fit_transform(mcTest_x.astype(np.float))

# Convert back to dataframe
mcTrain_x = pd.DataFrame(mcTrain_x, columns = cols, index = index_train)
mcTest_x = pd.DataFrame(mcTest_x, columns = cols, index = index_test)

# Construct & Run Neural Network

In [11]:
# Import PyTorch packages
import torch
from torch import nn
#from torchvision import datasets, transforms
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [12]:
# define list for results
results_ls = []

# store running loss
running_loss_res = []

# Where we will store correct/incorrect classifications
incorrect_ls = []
correct_ls = []

# Leave-one-out-cross-fold validation function - the for loop will iterate through the dataset, removing one sample (patient)
# at a time in order to create k training and test datasets (where k = number of total samples) always with one sample missing
for index in range (0, 242):
#for index in range (0, 212): # 212 observations when we downsample healthy patients 
    # X - features; 
    mcTrain_xy_drop = mcTrain_x.drop(mcTrain_x.index[index]) # add 'drop'suffix so we can differentiate the df with index and the array that will be created in next line
    mcTrain_xy = np.array(mcTrain_xy_drop, dtype = "float32")
    
    # y - target/outputs
    mcTrain_yz_drop = mcTrain_y.drop(mcTrain_y.index[index]) 
    mcTrain_yz = np.array(mcTrain_yz_drop, dtype = "float32")
    
    # reformat into tensors
    xb = torch.from_numpy(mcTrain_xy)
    yb = torch.from_numpy(mcTrain_yz)
    
    # squeeze - function is used when we want to remove single-dimensional entries from the shape of an array.
    yb = yb.squeeze(1) 
    
    # subset the equivalent test set
    mcTrain_test_x_drop = mcTrain_x.iloc[[index]] # add 'drop'suffix so we can differentiate the df with index and the array that will be created in next line
    mcTrain_test_x = np.array(mcTrain_test_x_drop, dtype = "float32")
            
    # y - targets/outputs
    mcTrain_test_y_drop = mcTrain_y.iloc[[index]]
    mcTrain_test_y = np.array(mcTrain_test_y_drop, dtype = "float32")
        
    # Convert arrays into tensors
    test_xb = torch.from_numpy(mcTrain_test_x)
    test_yb = torch.from_numpy(mcTrain_test_y)
    
    # Define the batchsize
    batch_size = 32

    # Combine the arrays
    trainloader = TensorDataset(xb, yb)
    
    # Training Loader
    trainloader = DataLoader(trainloader, batch_size, shuffle=True)
    
    ## Build the Model and define hyperparameters
    
    # Set parameters for grid search
    #lrs = [1e-2, 1e-3, 1e-4]
    #epochs = [100, 150, 200, 250]
    
    # summarize experiment with changed parameters
    summary = ('Hidden Layers: 50, LR: 0.10, Epochs: 100')
    
    # Define the model with hidden layers
    model = nn.Sequential(nn.Linear(157, 50),
                          nn.ReLU(),
                          nn.Linear(50, 7))
                      
    # Set Stoachastic Gradient Descent Optimizer and the learning rate
    #optimizer = optim.SGD(model.parameters(), lr=0.003)

    # Set Adam optimizer: similar to stochastic gradient descent, but uses momentum which can speed up the actual fitting process, and it also adjusts the learning rate for each of the individual parameters in the model
    optimizer = optim.Adam(model.parameters(), lr=0.01,  weight_decay=0.01) # we can also change momentum parameter

    # loss function
    criterion = nn.CrossEntropyLoss() #don't use with softmax or sigmoid- PyTorch manual indicates "This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class."
    
    # Set epochs - number of times the entire dataset will pass through the network
    epochs = 100
    for e in range(epochs):
        # Define running loss as 0
        running_loss = 0
        
        # Run the model for each xb, yb in the trainloader. For the number of epochs specified, the 
        for xb, yb in trainloader:
            # clear gradients - otherwise they are stored
            optimizer.zero_grad()
            # Training pass
            output = model.forward(xb)
            # caluclate loss calculated from the model output compared to the labels
            loss = criterion(output, yb.long()) 
            # backpropagate the loss
            loss.backward()
            # step function to update the weights
            optimizer.step()
        
            running_loss += loss.item() # loss.item() gets the scalar value held in the loss. 
            # += function: Adds the running_loss (0) with loss.item and assigns back to running_loss
        else:
            print("Epoch {}/{}, Training loss: {:.5f}".format(e+1, epochs, running_loss/len(trainloader)))
            
            # append all running_losses
            running_loss_res.append(running_loss/len(trainloader))
            
            # plotting
            #plt.plot(running_loss_res)
            #plt.title("Training Curve")
            #plt.xlabel("Iterations")
            #plt.ylabel("Loss")
            #plt.show()
            
            #print(running_loss_res)
            
            # I think next steps where the plotting is concerned, is to ensure that these running losses are getting averaged
            # across all patients? Additionally, is there a way to plot the validation loss...I'm not sure, I think I need to see how this works.
            
            #https://discuss.pytorch.org/t/is-my-implementation-of-calculating-training-and-validation-loss-correct-after-each-iteration-they-are-nearly-same/26793
            # We would be plotting the loss from the test - preducted 
            
    # Apply the model to the testing dataset
    # Thus will enable us to see the predictions for each class
    ps = model(test_xb)
    #print('Network Probabilities', ps)
    
    # Obtain the top prediction
    top_p, top_class = ps.topk(1, dim=1)
    #print('top prediction', top_p)
    #print('true vals', test_yb[:10])
        
    # Drop the grad by using detach
    top_p = top_p.detach().numpy()
    top_class = top_class.detach().numpy()

    # convert to integers
    top_class = top_class.astype(np.int)
    test_yb = test_yb.numpy()
    test_yb = test_yb.astype(np.int)
    
    #print('top class', top_class[:10])
    #print('prediction:', top_class)
    #print('true:', test_yb)
                
    # compare top_class to test_yb
    if top_class == test_yb:                
        results = 1 # prediction and true value are equal
    else: 
        results = 0
    
    # Create if-else statements to identify which classes are being classified correctly/incorrectly
    if results == 0:
        incorrect = test_yb
    else: 
        incorrect = np.array([[999]], dtype=int)
        
    if results == 1:
        correct = test_yb
    else: 
        correct = np.array([[999]], dtype=int)
    #print('Results:', results)
    
    # plotting
    #plt.plot(running_loss_res)
    #plt.title("Training Curve")
    #plt.xlabel("Iterations")
    #plt.ylabel("Loss")
    #plt.show()
            
            #print(running_loss_res)
    
    results_ls.append(results)
    incorrect_ls.append(incorrect)
    correct_ls.append(correct)
    print(results_ls) 

Epoch 1/100, Training loss: 1.89800
Epoch 2/100, Training loss: 1.89578
Epoch 3/100, Training loss: 1.84953
Epoch 4/100, Training loss: 1.84796
Epoch 5/100, Training loss: 1.87505
Epoch 6/100, Training loss: 1.84217
Epoch 7/100, Training loss: 1.83417
Epoch 8/100, Training loss: 1.82343
Epoch 9/100, Training loss: 1.83309
Epoch 10/100, Training loss: 1.82779
Epoch 11/100, Training loss: 1.81984
Epoch 12/100, Training loss: 1.81620
Epoch 13/100, Training loss: 1.84215
Epoch 14/100, Training loss: 1.82339
Epoch 15/100, Training loss: 1.84190
Epoch 16/100, Training loss: 1.81783
Epoch 17/100, Training loss: 1.80878
Epoch 18/100, Training loss: 1.79176
Epoch 19/100, Training loss: 1.80627
Epoch 20/100, Training loss: 1.78756
Epoch 21/100, Training loss: 1.79235
Epoch 22/100, Training loss: 1.76798
Epoch 23/100, Training loss: 1.76999
Epoch 24/100, Training loss: 1.77910
Epoch 25/100, Training loss: 1.79540
Epoch 26/100, Training loss: 1.75958
Epoch 27/100, Training loss: 1.77483
Epoch 28/1

Epoch 24/100, Training loss: 1.78294
Epoch 25/100, Training loss: 1.78576
Epoch 26/100, Training loss: 1.74322
Epoch 27/100, Training loss: 1.73105
Epoch 28/100, Training loss: 1.74912
Epoch 29/100, Training loss: 1.74573
Epoch 30/100, Training loss: 1.75065
Epoch 31/100, Training loss: 1.70365
Epoch 32/100, Training loss: 1.69919
Epoch 33/100, Training loss: 1.71595
Epoch 34/100, Training loss: 1.67157
Epoch 35/100, Training loss: 1.68599
Epoch 36/100, Training loss: 1.69455
Epoch 37/100, Training loss: 1.63979
Epoch 38/100, Training loss: 1.65188
Epoch 39/100, Training loss: 1.64060
Epoch 40/100, Training loss: 1.60295
Epoch 41/100, Training loss: 1.57075
Epoch 42/100, Training loss: 1.58654
Epoch 43/100, Training loss: 1.62047
Epoch 44/100, Training loss: 1.55855
Epoch 45/100, Training loss: 1.54317
Epoch 46/100, Training loss: 1.51131
Epoch 47/100, Training loss: 1.50393
Epoch 48/100, Training loss: 1.47765
Epoch 49/100, Training loss: 1.46672
Epoch 50/100, Training loss: 1.44781
E

Epoch 51/100, Training loss: 1.66452
Epoch 52/100, Training loss: 1.64918
Epoch 53/100, Training loss: 1.65409
Epoch 54/100, Training loss: 1.61853
Epoch 55/100, Training loss: 1.62215
Epoch 56/100, Training loss: 1.60194
Epoch 57/100, Training loss: 1.59141
Epoch 58/100, Training loss: 1.57667
Epoch 59/100, Training loss: 1.61415
Epoch 60/100, Training loss: 1.57735
Epoch 61/100, Training loss: 1.56277
Epoch 62/100, Training loss: 1.54317
Epoch 63/100, Training loss: 1.57412
Epoch 64/100, Training loss: 1.52794
Epoch 65/100, Training loss: 1.55802
Epoch 66/100, Training loss: 1.46572
Epoch 67/100, Training loss: 1.45840
Epoch 68/100, Training loss: 1.49126
Epoch 69/100, Training loss: 1.44964
Epoch 70/100, Training loss: 1.43412
Epoch 71/100, Training loss: 1.41965
Epoch 72/100, Training loss: 1.39402
Epoch 73/100, Training loss: 1.37696
Epoch 74/100, Training loss: 1.47308
Epoch 75/100, Training loss: 1.40516
Epoch 76/100, Training loss: 1.36113
Epoch 77/100, Training loss: 1.37409
E

Epoch 73/100, Training loss: 1.28604
Epoch 74/100, Training loss: 1.29089
Epoch 75/100, Training loss: 1.17726
Epoch 76/100, Training loss: 1.15538
Epoch 77/100, Training loss: 1.07298
Epoch 78/100, Training loss: 1.10960
Epoch 79/100, Training loss: 1.10484
Epoch 80/100, Training loss: 1.15668
Epoch 81/100, Training loss: 1.17390
Epoch 82/100, Training loss: 1.06117
Epoch 83/100, Training loss: 1.05144
Epoch 84/100, Training loss: 1.04029
Epoch 85/100, Training loss: 1.07628
Epoch 86/100, Training loss: 1.08438
Epoch 87/100, Training loss: 1.03868
Epoch 88/100, Training loss: 0.99199
Epoch 89/100, Training loss: 1.05790
Epoch 90/100, Training loss: 1.03509
Epoch 91/100, Training loss: 1.00751
Epoch 92/100, Training loss: 1.01790
Epoch 93/100, Training loss: 0.95806
Epoch 94/100, Training loss: 0.93226
Epoch 95/100, Training loss: 0.93455
Epoch 96/100, Training loss: 0.97416
Epoch 97/100, Training loss: 0.92397
Epoch 98/100, Training loss: 0.92349
Epoch 99/100, Training loss: 0.87797
E

Epoch 9/100, Training loss: 1.83545
Epoch 10/100, Training loss: 1.83561
Epoch 11/100, Training loss: 1.84288
Epoch 12/100, Training loss: 1.83213
Epoch 13/100, Training loss: 1.85011
Epoch 14/100, Training loss: 1.83984
Epoch 15/100, Training loss: 1.83913
Epoch 16/100, Training loss: 1.82075
Epoch 17/100, Training loss: 1.81650
Epoch 18/100, Training loss: 1.82876
Epoch 19/100, Training loss: 1.83946
Epoch 20/100, Training loss: 1.82165
Epoch 21/100, Training loss: 1.81745
Epoch 22/100, Training loss: 1.80781
Epoch 23/100, Training loss: 1.83084
Epoch 24/100, Training loss: 1.82139
Epoch 25/100, Training loss: 1.81191
Epoch 26/100, Training loss: 1.83710
Epoch 27/100, Training loss: 1.79353
Epoch 28/100, Training loss: 1.81007
Epoch 29/100, Training loss: 1.80058
Epoch 30/100, Training loss: 1.78451
Epoch 31/100, Training loss: 1.78695
Epoch 32/100, Training loss: 1.84517
Epoch 33/100, Training loss: 1.79084
Epoch 34/100, Training loss: 1.77086
Epoch 35/100, Training loss: 1.76193
Ep

...

Epoch 2/100, Training loss: 1.86713
Epoch 3/100, Training loss: 1.85801
Epoch 4/100, Training loss: 1.84669
Epoch 5/100, Training loss: 1.86302
Epoch 6/100, Training loss: 1.84825
Epoch 7/100, Training loss: 1.86888
Epoch 8/100, Training loss: 1.84724
Epoch 9/100, Training loss: 1.86876
Epoch 10/100, Training loss: 1.86260
Epoch 11/100, Training loss: 1.84266
Epoch 12/100, Training loss: 1.85256
Epoch 13/100, Training loss: 1.84475
Epoch 14/100, Training loss: 1.83744
Epoch 15/100, Training loss: 1.83934
Epoch 16/100, Training loss: 1.82920
Epoch 17/100, Training loss: 1.82647
Epoch 18/100, Training loss: 1.81637
Epoch 19/100, Training loss: 1.82915
Epoch 20/100, Training loss: 1.81944
Epoch 21/100, Training loss: 1.81025
Epoch 22/100, Training loss: 1.83072
Epoch 23/100, Training loss: 1.81947
Epoch 24/100, Training loss: 1.81422
Epoch 25/100, Training loss: 1.81478
Epoch 26/100, Training loss: 1.81297
Epoch 27/100, Training loss: 1.81251
Epoch 28/100, Training loss: 1.78685
Epoch 29/

Epoch 11/100, Training loss: 1.84395
Epoch 12/100, Training loss: 1.83942
Epoch 13/100, Training loss: 1.84689
Epoch 14/100, Training loss: 1.82308
Epoch 15/100, Training loss: 1.82597
Epoch 16/100, Training loss: 1.81300
Epoch 17/100, Training loss: 1.84245
Epoch 18/100, Training loss: 1.80515
Epoch 19/100, Training loss: 1.81766
Epoch 20/100, Training loss: 1.83611
Epoch 21/100, Training loss: 1.82157
Epoch 22/100, Training loss: 1.83551
Epoch 23/100, Training loss: 1.79863
Epoch 24/100, Training loss: 1.81834
Epoch 25/100, Training loss: 1.80069
Epoch 26/100, Training loss: 1.78011
Epoch 27/100, Training loss: 1.81351
Epoch 28/100, Training loss: 1.77373
Epoch 29/100, Training loss: 1.79594
Epoch 30/100, Training loss: 1.75941
Epoch 31/100, Training loss: 1.76020
Epoch 32/100, Training loss: 1.75324
Epoch 33/100, Training loss: 1.74250
Epoch 34/100, Training loss: 1.70021
Epoch 35/100, Training loss: 1.70671
Epoch 36/100, Training loss: 1.72258
Epoch 37/100, Training loss: 1.70655
E

Epoch 10/100, Training loss: 1.83369
Epoch 11/100, Training loss: 1.83426
Epoch 12/100, Training loss: 1.81309
Epoch 13/100, Training loss: 1.80483
Epoch 14/100, Training loss: 1.84791
Epoch 15/100, Training loss: 1.81151
Epoch 16/100, Training loss: 1.82643
Epoch 17/100, Training loss: 1.81791
Epoch 18/100, Training loss: 1.80596
Epoch 19/100, Training loss: 1.78515
Epoch 20/100, Training loss: 1.79228
Epoch 21/100, Training loss: 1.79206
Epoch 22/100, Training loss: 1.80430
Epoch 23/100, Training loss: 1.79343
Epoch 24/100, Training loss: 1.80845
Epoch 25/100, Training loss: 1.81588
Epoch 26/100, Training loss: 1.80084
Epoch 27/100, Training loss: 1.87817
Epoch 28/100, Training loss: 1.76405
Epoch 29/100, Training loss: 1.76077
Epoch 30/100, Training loss: 1.74519
Epoch 31/100, Training loss: 1.74744
Epoch 32/100, Training loss: 1.73749
Epoch 33/100, Training loss: 1.70675
Epoch 34/100, Training loss: 1.69515
Epoch 35/100, Training loss: 1.69807
Epoch 36/100, Training loss: 1.67073
E

Epoch 8/100, Training loss: 1.85417
Epoch 9/100, Training loss: 1.84818
Epoch 10/100, Training loss: 1.83183
Epoch 11/100, Training loss: 1.82576
Epoch 12/100, Training loss: 1.82041
Epoch 13/100, Training loss: 1.83139
Epoch 14/100, Training loss: 1.84130
Epoch 15/100, Training loss: 1.81686
Epoch 16/100, Training loss: 1.81005
Epoch 17/100, Training loss: 1.83344
Epoch 18/100, Training loss: 1.81020
Epoch 19/100, Training loss: 1.83921
Epoch 20/100, Training loss: 1.81714
Epoch 21/100, Training loss: 1.80086
Epoch 22/100, Training loss: 1.85045
Epoch 23/100, Training loss: 1.79215
Epoch 24/100, Training loss: 1.78618
Epoch 25/100, Training loss: 1.80765
Epoch 26/100, Training loss: 1.78636
Epoch 27/100, Training loss: 1.80239
Epoch 28/100, Training loss: 1.77577
Epoch 29/100, Training loss: 1.75969
Epoch 30/100, Training loss: 1.78177
Epoch 31/100, Training loss: 1.75267
Epoch 32/100, Training loss: 1.75953
Epoch 33/100, Training loss: 1.75987
Epoch 34/100, Training loss: 1.76176
Epo

Epoch 1/100, Training loss: 1.92005
Epoch 2/100, Training loss: 1.86601
Epoch 3/100, Training loss: 1.88653
Epoch 4/100, Training loss: 1.84866
Epoch 5/100, Training loss: 1.85774
Epoch 6/100, Training loss: 1.83625
Epoch 7/100, Training loss: 1.85401
Epoch 8/100, Training loss: 1.83765
Epoch 9/100, Training loss: 1.83294
Epoch 10/100, Training loss: 1.83032
Epoch 11/100, Training loss: 1.81879
Epoch 12/100, Training loss: 1.83651
Epoch 13/100, Training loss: 1.82877
Epoch 14/100, Training loss: 1.83237
Epoch 15/100, Training loss: 1.81094
Epoch 16/100, Training loss: 1.81669
Epoch 17/100, Training loss: 1.81472
Epoch 18/100, Training loss: 1.80086
Epoch 19/100, Training loss: 1.80195
Epoch 20/100, Training loss: 1.81879
Epoch 21/100, Training loss: 1.78020
Epoch 22/100, Training loss: 1.78358
Epoch 23/100, Training loss: 1.76411
Epoch 24/100, Training loss: 1.77286
Epoch 25/100, Training loss: 1.76505
Epoch 26/100, Training loss: 1.78056
Epoch 27/100, Training loss: 1.76848
Epoch 28/1

# **Average error over all the tests**

In [14]:
sum_tests = sum(results_ls) # 1 means correct - so we need to subtract 1 from the sum of the correct cases 
average_error = sum_tests/len(mcTrain_y)
average_error = 1 - average_error
#average_error # this is the same as the percent error below

0.8016528925619835

# **Determine LOOCV Mean Error**

In [16]:
percent_correct = sum(results_ls)
percent_correct = percent_correct/len(mcTrain_y)*100
percent_incorrect = 100 - percent_correct
#print('Percent Error', round(percent_incorrect, 1))

Percent Error 80.2


# **Incorrect Predictions**

In [18]:
## Remove the correct elements from the ls to faciliate transforming this list into a dataframe
# First, concatenate all incorrect list elements and format into dataframe
incorrect_res = np.concatenate(incorrect_ls)
incorrect_res = pd.DataFrame(incorrect_res)
incorrect_res.columns = ['diagnosis']
incorrect_res = incorrect_res[incorrect_res.diagnosis != 999] # 999 are the results that were correct - we remove these

# Count number of incorrect predictions by diagnosis
incorrect_pred = incorrect_res.groupby(['diagnosis']).size()
incorrect_pred = pd.DataFrame(incorrect_pred)
incorrect_pred.columns = ['Count']

# Convert the index to the first column and change the numebr to categorical variables
incorrect_pred.reset_index(level=0, inplace=True)
incorrect_pred['diagnosis'] = incorrect_pred['diagnosis'].map({0: 'HEA', 1: 'CRC', 2: 'ESCA', 3: 'HCC', 4: 'STAD', 5:'GBM', 6:'BRCA'})

# Add a column with the number of cases in each class
mcTrain_y['diagnosis'] = mcTrain_y['diagnosis'].map({0: 'HEA', 1: 'CRC', 2: 'ESCA', 3: 'HCC', 4: 'STAD', 5:'GBM', 6:'BRCA'})
class_size = mcTrain_y.groupby(['diagnosis']).size()
class_size = pd.DataFrame(class_size)
class_size.columns = ['Sample_n']

# bind class_size to the pred df diagnoses
incorrect_pred = pd.merge(incorrect_pred, class_size, how="left", on="diagnosis") 

# Calculate the percent error for each class
incorrect_pred['Count_Perc_Incorrect'] = incorrect_pred['Count']/incorrect_pred['Sample_n']
incorrect_pred['Count_Perc_Incorrect'] = incorrect_pred['Count_Perc_Incorrect'].multiply(100)

# **Correct Predictions**

In [19]:
## Remove the incorrect elements from the ls to faciliate transforming this list into a dataframe
# First, concatenate all incorrect list elements and format into dataframe
correct_res = np.concatenate(correct_ls)
correct_res = pd.DataFrame(correct_res)
correct_res.columns = ['diagnosis']
correct_res = correct_res[correct_res.diagnosis != 999] # 999 are the results that were incorrect - we remove these

# Count number of correct predictions by diagnosis
correct_pred = correct_res.groupby(['diagnosis']).size()
correct_pred = pd.DataFrame(correct_pred)
correct_pred.columns = ['Count']

# Convert the index to the first column and change the numebr to categorical variables
correct_pred.reset_index(level=0, inplace=True)
correct_pred['diagnosis'] = correct_pred['diagnosis'].map({0: 'HEA', 1: 'CRC', 2: 'ESCA', 3: 'HCC', 4: 'STAD', 5:'GBM', 6:'BRCA'})

# Add a column with the number of cases in each class
class_size = mcTrain_y.groupby(['diagnosis']).size()
class_size = pd.DataFrame(class_size)
class_size.columns = ['Sample_n']

# bind class_size to the pred df diagnoses
correct_pred = pd.merge(correct_pred, class_size, how="left", on="diagnosis") 

# Calculate the percent correct for each class
correct_pred['Count_Perc_Correct'] = correct_pred['Count']/correct_pred['Sample_n']
correct_pred['Count_Perc_Correct'] = correct_pred['Count_Perc_Correct'].multiply(100)

# **Save Predictions**

In [20]:
# convert float to string in order to save as a txt file
percent_error = '     Percent Error'
percent_error = str(percent_error)
percent_incorrect = str(percent_incorrect)

# save the pre-specified summary of the experiment which includes parameter spe$
summary = summary + percent_error + percent_incorrect

#Feb 21, 2020 Test
print(summary)
print(correct_pred)

Hidden Layers: 50, LR: 0.10, Epochs: 100     Percent Error80.16528925619835     Percent Error80.16528925619835
  diagnosis  Count  Sample_n  Count_Perc_Correct
0       HEA     28        61           45.901639
1       CRC      5        45           11.111111
2       HCC     15        49           30.612245
